# Summarizing (w/ GPT-2)

## Packages

In [1]:
from utils.json_utils import read_json
from datasets import Dataset
import pandas as pd

/home/pauli/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tokenizing

In [2]:
nonTokenizedWsubj_docs = read_json("18_docs_with_subjects_non_coref.json")
#nonTokenizedWsubj_docs

In [3]:
varied_set_adjectives = read_json("14_varied_set_adjectives_definitions.json")
#varied_set_adjectives

In [4]:
training_sents = []
for adj in varied_set_adjectives:
    training_sents.append("Someone can be described as " + adj + ": " + varied_set_adjectives[adj])

## Transformer

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

2023-01-23 23:05:12.349620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 23:05:12.540630: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-23 23:05:12.540643: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-23 23:05:13.141959: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [6]:
model_checkpoint = "gpt2"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [8]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [9]:
tokenize_fn = lambda doc : tokenizer(
        doc,
        truncation=True,
        max_length=1024
    )

tokenized_dataset = list(map(tokenize_fn, training_sents))

In [10]:
tokenized_dataset = pd.DataFrame(tokenized_dataset)
tokenized_dataset = Dataset.from_pandas(tokenized_dataset)

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint, pad_token_id=tokenizer.eos_token_id)

In [12]:
device = f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"

In [13]:
training_args = TrainingArguments(
    f"{model_checkpoint}-finetuned-variedadj-black-clover",
    num_train_epochs=100,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,    
    learning_rate=2e-5,
    weight_decay=0.01,
    do_eval=True, # eval en validation set
    evaluation_strategy="steps", # eval en validation set
    eval_steps=100,
    save_steps=100, # checkpoint model every 500 steps
    logging_dir='./logs', # logging
    logging_strategy="steps",
    logging_steps=1,
    fp16=False, # float16 en training (only on CUDA)
    push_to_hub=False,
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset, #.select(range(0, 128)),
    eval_dataset=tokenized_dataset, #.select(range(0, 128)),
)

In [15]:
train_output = trainer.train()

/home/pauli/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50
  Num Epochs = 100
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  0%|          | 1/500 [00:01<15:17,  1.84s/it]

{'loss': 4.9619, 'learning_rate': 1.9960000000000002e-05, 'epoch': 0.2}


  0%|          | 2/500 [00:03<14:05,  1.70s/it]

{'loss': 4.8097, 'learning_rate': 1.9920000000000002e-05, 'epoch': 0.4}


  1%|          | 3/500 [00:04<13:30,  1.63s/it]

...

 16%|█▌        | 81/500 [02:09<12:05,  1.73s/it]

{'loss': 0.9679, 'learning_rate': 1.6760000000000002e-05, 'epoch': 16.2}


 16%|█▋        | 82/500 [02:10<12:13,  1.76s/it]

{'loss': 0.9699, 'learning_rate': 1.672e-05, 'epoch': 16.4}


KeyboardInterrupt: 

In [ ]:
def generate(
    prompt=None, max_length=1024, max_new_tokens=20, greedy=True, model=model, tokenizer=tokenizer, device=device
):
    """None stands for beggining of sequence.
    NOTE si bien parece que GPT2 puede generar a partir de BOS token, la 
    documentacion es poco clara. Ademas hicimos nuestro finetuning sin BOS token.
    Entonces solo vamos a usar la funcion pasandole un contexto.

    Ver:
    https://github.com/huggingface/transformers/issues/3311#issuecomment-601264426
    https://github.com/openai/gpt-2/blob/a74da5d99abaaba920de8131d64da2862a8f213b/src/generate_unconditional_samples.py#L60
    """
    do_sample = False if greedy else True
    # model.eval() to set dropout and batch normalization layers to evaluation 
    # mode before running inference
    if prompt:
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
        model.eval()
        outputs = model.generate(input_ids, do_sample=do_sample, max_new_tokens=max_new_tokens)
    else:
        model.eval()
        outputs = model.generate(do_sample=do_sample, max_new_tokens=max_new_tokens)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

## Summarize

In [ ]:
generate("Microsoft's CEO is ", greedy=False)

["Microsoft's CEO is \xa0familiar with the state of the smartphone industry - how we learn.\nToday's post looks"]

In [ ]:
import re

In [ ]:
non_greedy_generated_descriptions = []
for doc in nonTokenizedWsubj_docs:
    for subject in doc["subjects"]:
        generated_doc = generate(doc["doc"] + " " + subject + " can be described as ", max_length=1024, greedy=False)
        if len(re.split(doc["doc"], generated_doc[0])) >= 2:
            generated_desc = re.sub(r"\s+", " ", (re.split(r"[.;:!?]",list(map(lambda x : x.strip(), re.split(doc["doc"], generated_doc[0])))[1]))[0])
            non_greedy_generated_descriptions.append(generated_desc)

In [ ]:
greedy_generated_descriptions = []
for doc in nonTokenizedWsubj_docs[:10]:
    for subject in doc["subjects"]:
        generated_doc = generate(doc["doc"] + " " + subject + " can be described as ", max_length=1024, greedy=True)
        if len(re.split(doc["doc"], generated_doc[0])) >= 2:
            generated_desc = re.sub(r"\s+", " ", (re.split(r"[.;:!?]",list(map(lambda x : x.strip(), re.split(doc["doc"], generated_doc[0])))[1]))[0])
            greedy_generated_descriptions.append(generated_desc)

In [ ]:
re.split("\.", "hol.")

['hol', '']

In [ ]:
non_greedy_generated_descriptions

['Yuno can be described as 『Darkness of the Shadow』',
 'Asta can be described as urchin-like',
 'Lily can be described as 『I Am Jin',
 'Yuno can be described as very quiet and composed',
 'Orsi can be described as 『Kung Fu Artist』or even is the "Chosen One of Magic Kingdom"',
 'Asta can be described as disappointed, even annoyed with the way the boys and girls fare',
 'Lily can be described as ~~very good~~',
 'Finral can be described as iced, as he does little but sing, as he is not strong and is often not present with',
 'Nozel can be described as a calm and calm type of man, whose calmness means at ease',
 'Gordon can be described as ________',
 'Rill can be described as 仙平巟翼九, a common form of Chinese 来',
 'Jack can be described as a charismatic man of the heart whose personal charisma can rival that of a high priest',
 'Dorothy can be described as 『Little Evil』',
 'Yami can be described as pessimistic and pessimistic',
 'Fuegoleon can be described as a cunning sorcerer who was abl

In [ ]:
greedy_generated_descriptions

['Yuno can be described as as strong as a lion, and has a strong sense of smell',
 'Asta can be described as a kind of devilish, but not evil',
 'Lily can be described as a kind of devilish spirit',
 'Yuno can be described as a very good student, and he is able to learn a lot from his classmates',
 'Orsi can be described as a very good student, and is very knowledgeable about the Magic Knights',
 'Asta can be described as a very strong and strong man',
 'Lily can be described as very cheerful and cheerful',
 'Finral can be described as a strong and experienced fighter',
 'Nozel can be described as as strong as he is, and he is able to defeat Sekke with ease',
 'Gordon can be described as a strong man',
 'Rill can be described as a strong and experienced fighter',
 'Jack can be described as a strong man',
 'Dorothy can be described as a strong woman',
 'Yami can be described as a strong and powerful man',
 'Fuegoleon can be described as a strong and powerful magician',
 'Yuno can be des